In [7]:
# -*- coding: utf-8 -*-
#

# pandas used for reading data
import pandas as pd

#pickle for dumping(saving) data in a file
import pickle

import tensorflow as tf

#Reading dataset
dataset = pd.read_csv("transliteration.txt",delimiter = "\t",header=None,encoding='utf-8',na_filter = False)


In [8]:
#Splitting English words in X and Hindi words in y
X = dataset.iloc[:,0]
y = dataset.iloc[:,-1]

#importing the preprocessed data 
#The preprocessing is done in a file named Data_preprocessing
import Data_preprocessing

In [16]:
#source_int_text is the English words' processed vector. (Word is Converted to integer vector)
#target_int_text is the Hindi words' processed vector.
#source_vocab_to_int and  source_int_to_vocab are the English dictionaries
#target_vocab_to_int and  target_int_to_vocab are the Hindi dictionaries

source_int_text, target_int_text, source_vocab_to_int, target_vocab_to_int,source_int_to_vocab,target_int_to_vocab = Data_preprocessing.preprocess(X,y)

#encoder and decoder layers are defined in Layers 
#placeholders are defined in Model_Inputs
import Layers
import Model_Inputs


In [10]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_word_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):

#    Build the Sequence-to-Sequence model
#    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    enc_outputs, enc_states = Layers.encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = Model_Inputs.process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = Layers.decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_word_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [11]:
# initialising the parameters
display_step = 200

epochs = 60
batch_size = 30

rnn_size = 64
num_layers = 2

encoding_embedding_size = 50
decoding_embedding_size = 50

learning_rate = 0.001
keep_probability = 0.5


# path for saving the model
save_path = 'checkpoints/dev'


In [22]:
tf.compat.v1.disable_v2_behavior() 


Instructions for updating:
non-resource variables are not supported in the long term


In [24]:
def enc_dec_model_inputs():
#   English Inputs
    inputs = tf.compat.v1.placeholder(tf.int32, [None, None], name='input')
    
#   Hindi Inputs
    targets = tf.compat.v1.placeholder(tf.int32, [None, None], name='targets') 

#   Length of Hindi words
    target_sequence_length = tf.compat.v1.placeholder(tf.int32, [None], name='target_sequence_length')
    
    #computes the maximum of all elements across the dimension
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len

def hyperparam_inputs():
    
    #learning rate
    lr_rate = tf.compat.v1.placeholder(tf.float32, name='lr_rate')
    
    #probability of dropout
    keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob

def process_decoder_input(target_data, target_vocab_to_int, batch_size):

#    Preprocess target data for encoding
#    :return: Preprocessed target data
        
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    #Take the batch of Hindi target
    #tf.strided_slice(input_,begin,end,strides=None)
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    
    #Concat every word of the Hindi target with <GO> id 
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat	

In [25]:
    
#initialising the graph
train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length =enc_dec_model_inputs()
#   taking learning rate and probability of drop out layer    
    lr, keep_prob = hyperparam_inputs()

In [28]:
train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
training_logits = tf.identity(train_logits.rnn_output, name='logits')
inference_logits = tf.identity(inference_logits.sample_id, name='predictions')


masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')


AttributeError: module 'tensorflow' has no attribute 'contrib'